# Instalando dependências

In [1]:
! pip install transformers datasets accelerate torchinfo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.0 MB/s eta 0:00:00


# Preparando dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/datasets/B2W-Reviews01.csv")

<ipython-input-3-6ce9e652ffa3>:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/MyDrive/datasets/B2W-Reviews01.csv")


In [4]:
df.head(1)

,submission_date,reviewer_id,product_id,product_name,product_brand,site_category_lv1,site_category_lv2,review_title,overall_rating,recommend_to_a_friend,review_text,reviewer_birth_year,reviewer_gender,reviewer_state
0,2018-01-01 00:11:28,d0fb1ca69422530334178f5c8624aa7a99da47907c44de...,132532965,Notebook Asus Vivobook Max X541NA-GO472T Intel...,NaN,Informática,Notebook,Bom,4,Yes,Estou contente com a compra entrega rápida o ú...,1958.0,F,RJ


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132373 entries, 0 to 132372
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   submission_date        132373 non-null  object 
 1   reviewer_id            132373 non-null  object 
 2   product_id             132373 non-null  object 
 3   product_name           132289 non-null  object 
 4   product_brand          40982 non-null   object 
 5   site_category_lv1      132367 non-null  object 
 6   site_category_lv2      128360 non-null  object 
 7   review_title           132071 non-null  object 
 8   overall_rating         132373 non-null  int64  
 9   recommend_to_a_friend  132355 non-null  object 
 10  review_text            129098 non-null  object 
 11  reviewer_birth_year    126389 non-null  float64
 12  reviewer_gender        128237 non-null  object 
 13  reviewer_state         128382 non-null  object 
dtypes: float64(1), int64(1), object(12)


In [19]:
df["overall_rating"] = df["overall_rating"] - 1

In [20]:
dataset = df[["review_text", "overall_rating"]].copy()

In [21]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132373 entries, 0 to 132372
Data columns (total 2 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   review_text     129098 non-null  object
 1   overall_rating  132373 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.0+ MB


In [22]:
dataset.dropna(inplace=True)

In [23]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 129098 entries, 0 to 132372
Data columns (total 2 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   review_text     129098 non-null  object
 1   overall_rating  129098 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.0+ MB


In [24]:
dataset.to_csv("dataset.csv", index=None)

In [25]:
from datasets import load_dataset
dataset_raw = load_dataset("csv", data_files="dataset.csv")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [26]:
dataset_raw = dataset_raw.rename_column("review_text", "sentence")
dataset_raw = dataset_raw.rename_column("overall_rating", "labels")

In [27]:
dataset_raw["train"][0]

{'sentence': 'Estou contente com a compra entrega rápida o único problema com as Americanas é se houver troca ou devolução do produto o consumidor tem problemas com espera.',
 'labels': 2}

In [28]:
dataset_split = dataset_raw["train"].train_test_split(test_size=0.35, seed=99)

In [29]:
dataset_split

DatasetDict({
    train: Dataset({
        features: ['sentence', 'labels'],
        num_rows: 83913
    })
    test: Dataset({
        features: ['sentence', 'labels'],
        num_rows: 45185
    })
})

# Instanciando tokenizer e modelo

In [30]:
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    AutoConfig,
    TrainingArguments,
    Trainer
)

In [31]:
model_id = "neuralmind/bert-base-portuguese-cased"
config = AutoConfig.from_pretrained(model_id)

In [32]:
config.id2label = {
    0: "Muito Negativo",
    1: "Negativo",
    2: "Neutro",
    3: "Positivo",
    4: "Muito Positivo"
}

In [33]:
config.label2id = {
    "Muito Negativo": 0,
    "Negativo": 1,
    "Neutro": 2,
    "Positivo": 3,
    "Muito Positivo": 4
}

In [34]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
base_model = AutoModelForSequenceClassification.from_pretrained(model_id, config=config)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
def tokenize_fn(batch):
  return tokenizer(batch["sentence"], truncation=True, padding="max_length", max_length=40)

In [36]:
tokenized_dataset = dataset_split.map(tokenize_fn, batched=True)

Map:   0%|          | 0/83913 [00:00<?, ? examples/s]

Map:   0%|          | 0/45185 [00:00<?, ? examples/s]

In [26]:
training_args = TrainingArguments(
    output_dir = "output_sa",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    logging_steps=150,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    push_to_hub=True,
    hub_model_id = "ramobertimbau-product-reviews-pt-br"
)

In [27]:
import numpy as np
from sklearn.metrics import f1_score
def compute_metrics(logits_and_labels):
  logits, labels = logits_and_labels
  predictions = np.argmax(logits, axis=-1)
  acc = np.mean(predictions==labels)
  f1 = f1_score(labels, predictions, average="macro")

  return {
      "accuracy": acc,
      "f1_score": f1
  }

In [28]:
trainer = Trainer(
    base_model,
    training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [29]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Score
1,0.863100,0.873007,0.621733,0.491815
2,0.802800,0.897009,0.620914,0.554796
3,0.692400,0.966356,0.619210,0.557672


TrainOutput(global_step=31470, training_loss=0.8075866708916105, metrics={'train_runtime': 3631.984, 'train_samples_per_second': 69.312, 'train_steps_per_second': 8.665, 'total_flos': 5174773286386320.0, 'train_loss': 0.8075866708916105, 'epoch': 3.0})

In [ ]:
trainer.push_to_hub()

In [35]:
trainer.save_model("my-model")

In [30]:
! pip install --upgrade huggingface_hub

In [31]:
from huggingface_hub import notebook_login
notebook_login()

In [43]:
# testando

from transformers import pipeline

pipe = pipeline(
    "text-classification",
    "my-model"
)

In [57]:
pipe("O produto poderia ser muito melhor")

[{'label': 'Negativo', 'score': 0.889604389667511}]

In [65]:
model_final = AutoModelForSequenceClassification.from_pretrained("my-model")
tokenizer = AutoTokenizer.from_pretrained("my-model")

In [61]:
from torchinfo import summary
summary(model_final)

Layer (type:depth-idx)                                  Param #
BertForSequenceClassification                           --
├─BertModel: 1-1                                        --
│    └─BertEmbeddings: 2-1                              --
│    │    └─Embedding: 3-1                              22,881,792
│    │    └─Embedding: 3-2                              393,216
│    │    └─Embedding: 3-3                              1,536
│    │    └─LayerNorm: 3-4                              1,536
│    │    └─Dropout: 3-5                                --
│    └─BertEncoder: 2-2                                 --
│    │    └─ModuleList: 3-6                             85,054,464
│    └─BertPooler: 2-3                                  --
│    │    └─Linear: 3-7                                 590,592
│    │    └─Tanh: 3-8                                   --
├─Dropout: 1-2                                          --
├─Linear: 1-3                                           3,845
Total params: 10

In [66]:
tokenizer.push_to_hub(repo_id="ramonmedeiro1/bertimbau-products-reviews-pt-br", create_pr=True)

CommitInfo(commit_url='https://huggingface.co/ramonmedeiro1/bertimbau-products-reviews-pt-br/commit/68bee87b351f0fa2558769957c8690e29a9bc11c', commit_message='Upload tokenizer', commit_description='', oid='68bee87b351f0fa2558769957c8690e29a9bc11c', pr_url='https://huggingface.co/ramonmedeiro1/bertimbau-products-reviews-pt-br/discussions/2', pr_revision='refs/pr/2', pr_num=2)

In [64]:
model_final.push_to_hub(repo_id="ramonmedeiro1/bertimbau-products-reviews-pt-br", create_pr=True)

CommitInfo(commit_url='https://huggingface.co/ramonmedeiro1/bertimbau-products-reviews-pt-br/commit/de95648a3a7cfe3a294909f3bac661d81f5944ce', commit_message='Upload BertForSequenceClassification', commit_description='', oid='de95648a3a7cfe3a294909f3bac661d81f5944ce', pr_url='https://huggingface.co/ramonmedeiro1/bertimbau-products-reviews-pt-br/discussions/1', pr_revision='refs/pr/1', pr_num=1)